In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np
import pandas as pd
import torch.nn as nn
import re
import warnings

In [ ]:
!pip3 install rouge-score
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
def find_max_similarity(answer):
  segments = [s.strip() for s in re.split('[,.!?]', answer) if len(s.strip().split()) >= 2]
  max_score, s1, s2 = 0, "", ""
  if len(segments) > 0:
    for i in range(len(segments)):
      for j in range(i + 1, len(segments)):
        score = scorer.score(segments[i], segments[j])['rougeL'].fmeasure
        if score > max_score: max_score, s1, s2 = score, segments[i], segments[j]
  return (max_score, s1, s2)

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7b24678d6b31873d595ed5d7bd9bab961f46494f41ac90d32c9078f799baddf5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from google.colab import drive
drive.mount ("/content/drive")
import os
os.chdir("/content/drive/MyDrive/TEST")
os.environ['WANDB_MODE'] = 'disabled'
model_name="./MODEL/qa_model2"
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_QA = pd.read_csv("QA_pairs.csv")
df_QA2 = pd.read_csv("QA_pairs_simple.csv")

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

In [ ]:
def predict(question, answer, model, tokenizer):
    text = f"Question: {question} Answer: {answer}"
    inputs = tokenizer(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        model = model.cuda()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits.float(), dim=-1)
    label, confidence = int(predictions.argmax().item()), round(predictions.max().item(),3)
    return label, confidence

In [ ]:
def clean_text(text):
   text = text.replace('\n', ' ')
   text = text.replace('<userStyle>Normal</userStyle>', '')
   text = ' '.join(text.split())
   return text
combined_df = pd.DataFrame({
   'question': df_QA['Question'],
   'answer_LLAMA': df_QA['LLAMA_Finetuned'].apply(clean_text),
   'summary_bart': df_QA2['answers_simple1_bart_large_cnn'].apply(clean_text),
   'summary_llama': df_QA2['answers_simple2_llama'].apply(clean_text)
})
def get_final_label(question, answer):
   label, _ = predict(question, answer, model, tokenizer)
   max_score, _, _ = find_max_similarity(answer)
   if label == 1 and max_score < 0.65:
       return 1
   return 0
combined_df['label_LLAMA'] = [
   get_final_label(q, a)
   for q, a in zip(combined_df['question'], combined_df['answer_LLAMA'])
]
combined_df['label_bart'] = [
   get_final_label(q, a)
   for q, a in zip(combined_df['question'], combined_df['summary_bart'])
]
combined_df['label_llama'] = [
   get_final_label(q, a)
   for q, a in zip(combined_df['question'], combined_df['summary_llama'])
]

In [ ]:
len(combined_df[combined_df['label_LLAMA'] == 1]), len(combined_df[combined_df['label_bart'] == 1]), len(combined_df[combined_df['label_llama'] == 1])

(5, 93, 84)

In [ ]:
combined_df.to_csv("QA_pairs_scoring_byfittedmodel3.csv",index=False)
combined_df

,question,answer_LLAMA,summary_bart,summary_llama,label_LLAMA,label_bart,label_llama
0,"Why is the ""Black Death"" mentioned so often in western history but not Chinese history even though it originated there?","The Black Death is a very famous event in European history. It was a major factor in the rise of the nation-states of Europe. It's not like that in China. The plague didn't have a major impact on the political landscape of China. The plague didn't even make it to China. It was in Europe, and it's not like that in Asia. The plague didn't have a major impact on the political landscape of China. The plague didn't even make it to China. If you're interested in reading more about the Black Death, I recommend the book *The Plague* by Robin Brooks. It's a great read, and it's very well written.<|eot_id|>","The ""Black Death"" originated in the Middle East and was introduced to Europe through the Silk Road. European records of the plague are limited and often biased. In Chinese culture, the plague is often seen as a natural disaster or a punishment from heaven.","The ""Black Death"" is a historical phenomenon that occurred in Europe and spread globally, but its significance in Chinese history is largely forgotten. Limited European records, cultural sensitivity, and a lack of European influence have contributed to this lack of documentation. Archaeological evidence is also scarce in China. However, Chinese historians have written about the plague in their books, and there are records of outbreaks in China during the 14th century. Despite these efforts, much more work needs to be done to fully understand the plague's historical significance in China.",0,1,1
1,How historically accurate is the show The Last Kingdom on Netflix?,"I've watched the first season of the show and I'm not sure if you are asking for historical accuracy, but I would say that it is mostly accurate, except for the characters. I am not sure if you are asking for accuracy in terms of the setting, but I am sure that the show is mostly accurate in terms of the setting. The show is set in the 9th century, and the show's setting is the 9th century, so it is accurate. The show is set in a very similar time period to the Viking Age, so it is accurate in terms of clothing, weapons, and the like. However, the show is set in a very different time period than the Viking Age, and so it is not accurate in terms of politics, religion, and culture. I'm not sure if you are asking for accuracy in terms of the characters, but I would say that the show is mostly accurate. The characters are mostly based on historical figures, but I don't know if they are accurate.<|eot_id|>","The Last Kingdom is a historical drama series based on the novels by Bernard Cornwell. While the show takes creative liberties with the past, it does draw inspiration from historical records and archaeological findings. Here's a breakdown of its historical accuracy.","The Last Kingdom, a historical drama series, is set in 9th-century England. It draws inspiration from historical records and archaeological findings, but takes creative liberties to create a compelling narrative. The show's timeline is generally accurate, with events taking place from 870-900 AD. The portrayal of historical events, such as the Roman withdrawal, Viking invasions, and Saxon kingdoms, is mostly accurate, but some artistic liberties are taken, including exaggerated Viking raids and a gradual Roman withdrawal.",0,1,1
2,"If there was a lack of food supply in the world, could we survive of of just vitamins and a high concentration of calories? Or do we HAVE to eat in order to live?","There's a lot of food out there. You just need to have a good supply of it. I know it sounds crazy, but I've seen people survive on very little food and still have a good supply of it. The problem is, you have to have a good supply of food to begin with. You can't just go out and find a good supply of food. You need to have a good supply of food before y